In [4]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
from selenium import webdriver
import pandas as pd

# Testing Phase

In [5]:
def getInput():
    print("PRESS 1 : For top tweets & posts\nPRESS 2 : For Latest tweets & posts\nPRESS 3 : For specific User posts")
    while True:
        choice = int(input("Enter your Choice: ")) 
        if choice==1:
            return "Top"
        elif choice==2:
            return "Latest"
        elif choice==3:
            return "People"
        else:
            print("Invalid Choice. Try again!!")
            
def track_css(user,driver,path):
    try:
        username = driver.find_element_by_css_selector(path)
        username.send_keys(user)
        username.send_keys(Keys.RETURN)
        return ["success",driver]
    except Exception as e:
        return ["error","error"]

def track_xpath(search_term,driver,path):
    try:
        search_input = driver.find_element_by_xpath(path)
        search_input.send_keys(search_term)
        search_input.send_keys(Keys.RETURN)
        return ["success",driver]
    except Exception as e:
        return ["error","error"]
def start(limit,driver):
    sleep_time = 3
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    curr = 0
    
    while scrolling and len(data)<=limit:
        
        post_element_xpath = '//div/div/article/div/div'
        page_cards = driver.find_elements_by_xpath(post_element_xpath)
        for card in page_cards:
            card = card.text
            card = card.split("\n")
            tweet = ' '.join(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(tweet)

        scroll_attempt = 0
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(sleep_time)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1
                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(sleep_time) # attempt another scroll
            else:
                last_position = curr_position
                break
        print("post_fectched:\t",len(data))
    print("post_fectched:\t",len(data))
    print("DONE")
    # close the web driver
    driver.close()
    return data

In [6]:


user = input('username: ')
my_password = getpass('Password: ')
tab = getInput()
search_term = "covid"

if(tab=="People"):
    search_term = input("Enter the correct username of that person, For e.g. -> narendramodi:\t\t")
    
else:   
    search_term = input('search term: ')

limit = int(input("How many post are required:\t"))

# create instance of web driver
driver = Chrome()

# navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()

user_path = '#layers > div:nth-child(2) > div > div > div > div > div > div.css-1dbjc4n.r-1awozwy.r-18u37iz.r-1pi2tsx.r-1777fci.r-1xcajam.r-ipm5af.r-g6jmlv > div.css-1dbjc4n.r-1867qdf.r-1wbh5a2.r-kwpbio.r-rsyp9y.r-1pjcn9w.r-1279nm1.r-htvplk.r-1udh08x > div > div > div.css-1dbjc4n.r-14lw9ot.r-6koalj.r-16y2uox.r-1wbh5a2 > div.css-1dbjc4n.r-16y2uox.r-1wbh5a2.r-1jgb5lz.r-1ye8kvj.r-13qz1uu > div.css-1dbjc4n.r-16y2uox.r-1wbh5a2.r-1dqxon3 > div > div.css-1dbjc4n.r-mk0yit.r-1f1sjgu.r-13qz1uu > label > div > div.css-1dbjc4n.r-18u37iz.r-16y2uox.r-1wbh5a2.r-1wzrnnt.r-1udh08x.r-xd6kpl.r-1pn2ns4.r-ttdzmv > div > input'
pwd_path = '#layers > div:nth-child(2) > div > div > div > div > div > div.css-1dbjc4n.r-1awozwy.r-18u37iz.r-1pi2tsx.r-1777fci.r-1xcajam.r-ipm5af.r-g6jmlv > div.css-1dbjc4n.r-1867qdf.r-1wbh5a2.r-kwpbio.r-rsyp9y.r-1pjcn9w.r-1279nm1.r-htvplk.r-1udh08x > div > div > div.css-1dbjc4n.r-14lw9ot.r-6koalj.r-16y2uox.r-1wbh5a2 > div.css-1dbjc4n.r-16y2uox.r-1wbh5a2.r-1jgb5lz.r-1ye8kvj.r-13qz1uu > div.css-1dbjc4n.r-16y2uox.r-1wbh5a2.r-1dqxon3 > div > div.css-1dbjc4n.r-mk0yit.r-13qz1uu > div > label > div > div.css-1dbjc4n.r-18u37iz.r-16y2uox.r-1wbh5a2.r-1wzrnnt.r-1udh08x.r-xd6kpl.r-1pn2ns4.r-ttdzmv > div.css-901oao.r-1awozwy.r-6koalj.r-37j5jr.r-1inkyih.r-16dba41.r-135wba7.r-bcqeeo.r-13qz1uu.r-qvutc0 > input'
search_path = '//input[@aria-label="Search query"]'

res = ["error",0]
while res[0]!="success":
    res = track_css(user,driver,user_path)
    sleep(1)

driver = res[1]
res = ["error",0]

while res[0]!="success":
    res = track_css(my_password,driver,pwd_path)
    sleep(1)

driver = res[1]
res = ["error",0]


while res[0]!="success":
    res = track_xpath(search_term,driver,search_path)
    sleep(1)
    
driver = res[1]
res = ["error",0]
while res[0]!="success": 
    sleep(1)
    try:
        driver.find_element_by_link_text(tab).click()
        res = ["success",driver]
    except Exception as e:
        res = ["error","error"]
driver = res[1]
if(tab=="People"):
    driver.get('https://www.twitter.com/{}'.format(search_term))

data = start(limit,driver)

PRESS 1 : For top tweets & posts
PRESS 2 : For Latest tweets & posts
PRESS 3 : For specific User posts
Enter your Choice: 1
search term: omicron
How many post are required:	100
post_fectched:	 9
post_fectched:	 24
post_fectched:	 39
post_fectched:	 51
post_fectched:	 63
post_fectched:	 73
post_fectched:	 81
post_fectched:	 90
post_fectched:	 99
post_fectched:	 108
post_fectched:	 108
DONE


In [7]:
dataset = []
for i in range(len(data)):
    dct = {"handle":"","username":"","text":"","retweets":"","likes":"","comments":""}
    post = data[i].split(" ")
    dct["text"],(dct["comments"],dct["retweets"],dct["likes"]) = " ".join(post[:-3]),post[-3:]
    
    for k in ["retweets","comments","likes"]:
        dct[k] = dct[k] if dct[k].isdigit() else 0
        
    temp = dct["text"].split("·")[0][:-1].split(" ")
    
    dct["handle"] = temp[-1]
    dct["username"] = "".join(temp[:-1])
    dct["text"] = " ".join(dct["text"].split("·")[1].split(" ")[3:])
    print(dct,"\n")
    dataset.append(tuple(dct.values()))

{'handle': '@GeorgeTakei', 'username': 'GeorgeTakei', 'text': 'folks: I’m begging you. Don’t do this to yourselves or your loved ones. The Omicron variant appears highly contagious, and it could wreak havoc among the unvaccinated. Set aside your politics and think of your families.', 'retweets': 0, 'likes': 0, 'comments': '399'} 

{'handle': '@ZubyMusic', 'username': 'ZUBY:', 'text': "Omicron variant' is basically a cold.  People are so weak and easily frightened these days. Literally afraid to breathe air, be around others, or catch the sniffles...   The pussification of Western society is complete.", 'retweets': 0, 'likes': 0, 'comments': '629'} 

{'handle': '@SSalud_mx', 'username': 'SALUDMéxico', 'text': ' Detecta México la primera muestra positiva a la variante #Ómicron   http://bit.ly/3ItCl5J', 'retweets': 0, 'likes': 0, 'comments': '231'} 

{'handle': '@Super70sSports', 'username': 'Super70sSports', 'text': '‘Omicron’ one more goddamn time.”', 'retweets': '392', 'likes': 0, 'com

In [8]:
with open('twitter_raw_data.csv', 'w', newline='', encoding='utf-8') as f:
    header = ["Handle",'UserName','Text', 'Retweets','Likes','Comments']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(dataset)

In [9]:
df = pd.read_csv("twitter_raw_data.csv")

In [10]:
df

,Handle,UserName,Text,Retweets,Likes,Comments
0,@GeorgeTakei,GeorgeTakei,folks: I’m begging you. Don’t do this to yours...,0,0,399
1,@ZubyMusic,ZUBY:,Omicron variant' is basically a cold. People ...,0,0,629
2,@SSalud_mx,SALUDMéxico,Detecta México la primera muestra positiva a ...,0,0,231
3,@Super70sSports,Super70sSports,‘Omicron’ one more goddamn time.”,392,0,62
4,@CassyWearsHeels,Cassandra,The WHO has reported that not one person has d...,0,0,128
...,...,...,...,...,...,...
103,@Row_tweets_,IDontThinkIKnow,Marshall doing a Gladys and ignoring health ad...,2,5,0
104,@imaanjoshi,imaanjoshi,"it. “We cared primarily – really, exclusively ...",13,40,1
105,@mugecevik,MugeCevik,Immune response Mutations in the omicron might...,22,258,2
106,@barbvee,BarbVee,to @GANyborg This is not looking good for the...,22,59,1


In [3]:
def getInput():
    print("PRESS 1 : For top tweets & posts\nPRESS 2 : For Latest tweets & posts\nPRESS 3 : For specific User posts")
    while True:
        choice = int(input("Enter your Choice: ")) 
        if choice==1:
            return "Top"
        elif choice==2:
            return "Latest"
        elif choice==3:
            return "People"
        else:
            print("Invalid Choice. Try again!!")
    

In [4]:
# application variables
user = input('username: ')
user = "Tushar69232214"
my_password = getpass('Password: ')
my_password = "667rias4781"
tab = getInput()
search_term = ""
if(tab=="People"):
    search_term = input("Enter the correct username of that person, For e.g. -> narendramodi:\t\t")
else:   
    search_term = input('search term: ')


# create instance of web driver
driver = Chrome()

# navigate to login screen
driver.get('https://www.twitter.com/login')
driver.maximize_window()

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys(user)

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(3)

# find search input and search for term
search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys(search_term)
search_input.send_keys(Keys.RETURN)
sleep(1)

# navigate to historical 'latest' tab
driver.find_element_by_link_text(tab).click()

if(tab=="People"):
    driver.get('https://www.twitter.com/'+search_term)
limit = int(input("How many post are required"))

username: tushar7452
Password: ········
PRESS 1 : For top tweets & posts
PRESS 2 : For Latest tweets & posts
PRESS 3 : For specific User posts
Enter your Choice: 1
search term: modi


In [ ]:
# ActionChains(driver).move_to_element(driver.find_element_by_xpath('//div[@data-testid="tweet"]').find_element_by_xpath('.//span[contains(text(), "@")]')).click

In [5]:

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling and len(data)<=10:
    print(len(data))
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
# driver.close()

0
5


In [ ]:
data

In [ ]:
new_data

In [ ]:
driver.close()

In [ ]:
data